# Sentiment Reputation Monitoring - Report di Consegna

Questo documento presenta il progetto completo per il monitoraggio della reputazione online tramite analisi del sentiment su testi provenienti da social media e feedback clienti.

Il sistema implementa una pipeline end-to-end che va dalla raccolta di feedback testuali fino alla visualizzazione di metriche operative su dashboard, passando per l'inferenza di sentiment analysis, il deployment containerizzato e il monitoring continuo.


## 1. Introduzione

### Contesto

Il progetto nasce dall'esigenza di un'azienda meccanica di monitorare in tempo reale la propria reputazione online. Ogni giorno arrivano centinaia di feedback: post su LinkedIn, recensioni su forum B2B, commenti su gruppi specializzati. Un cliente segnala rumorosità eccessiva su un modello di ingranaggio, un altro elogia i tempi di consegna. Come trasformare questo flusso continuo di informazioni non strutturate in insight operativi?

### Obiettivi

L'obiettivo principale è costruire un sistema che:

- Analizzi automaticamente il sentiment di testi provenienti da diverse fonti
- Esponga un'API REST per integrazione con altri sistemi
- Tracci metriche operative per monitoring continuo
- Rilevi automaticamente quando la distribuzione dei feedback cambia (concept drift)
- Sia deployabile facilmente con Docker e orchestrato con CI/CD

### Repository GitHub

Il codice completo del progetto è disponibile su GitHub:

**https://github.com/yourusername/sentiment-reputation**

Il repository include tutto il codice sorgente, i test, la configurazione Docker, i workflow CI/CD e la documentazione completa.


## 2. Setup Ambiente

Per eseguire questo notebook su Google Colab, installiamo le dipendenze essenziali. Nota: in produzione si usa un ambiente virtuale Python e un Dockerfile completo, ma per questa dimostrazione ci limitiamo ai pacchetti necessari per l'inferenza e i test API.


In [ ]:
!pip install fastapi uvicorn transformers torch requests prometheus-client pytest -q


In [ ]:
# Creiamo un file di esempio con alcuni campioni per la dimostrazione
import json

samples = [
    {"text": "I love this product! It's amazing and works perfectly.", "label": "positive"},
    {"text": "This is terrible. I'm very disappointed.", "label": "negative"},
    {"text": "The product arrived on time. It's okay.", "label": "neutral"},
    {"text": "Best purchase ever! Highly recommend to everyone.", "label": "positive"},
    {"text": "Not satisfied with the quality. Poor service.", "label": "negative"}
]

with open("samples.jsonl", "w") as f:
    for sample in samples:
        f.write(json.dumps(sample) + "\n")

print(f"Creato file samples.jsonl con {len(samples)} campioni")


## 3. Dimostrazione Inferenza

Il modello utilizzato è `cardiffnlp/twitter-roberta-base-sentiment-latest`, un RoBERTa base fine-tunato su tweet per sentiment analysis. Questo modello è particolarmente adatto per analizzare linguaggio informale e slang tipico dei social media.

Carichiamo il modello e testiamo alcune predizioni su esempi realistici.


In [ ]:
from transformers import pipeline

# Caricamento del modello (la prima volta può richiedere qualche minuto per il download)
print("Caricamento modello...")
sentiment_pipeline = pipeline(
    "sentiment-analysis",
    model="cardiffnlp/twitter-roberta-base-sentiment-latest",
    return_all_scores=False,
    max_length=512,
    truncation=True
)
print("Modello caricato con successo!")


In [ ]:
# Funzione per normalizzare le label del modello
def normalize_label(label):
    """Normalizza le label del modello in formato standardizzato."""
    label_lower = label.lower()
    if "negative" in label_lower or label_lower == "label_0":
        return "negative"
    elif "neutral" in label_lower or label_lower == "label_1":
        return "neutral"
    elif "positive" in label_lower or label_lower == "label_2":
        return "positive"
    return label_lower

# Test su alcuni esempi
test_texts = [
    "Ottimi ingranaggi, consegna puntuale. Consigliato!",
    "Prodotto arrivato rotto. Assistenza clienti non risponde.",
    "Ingranaggi funzionano, nulla di eccezionale ma ok per il prezzo."
]

print("Risultati predizioni:\n")
for text in test_texts:
    result = sentiment_pipeline(text)
    label_raw = result[0]["label"]
    score = result[0]["score"]
    label = normalize_label(label_raw)
    
    print(f"Testo: {text}")
    print(f"Sentiment: {label} (confidenza: {score:.2%})")
    print()


### Come interpretare i risultati

Il modello restituisce una label (negative, neutral, positive) e uno score di confidenza tra 0 e 1. Uno score alto (es. > 0.9) indica che il modello è molto sicuro della sua predizione. Uno score basso (es. < 0.6) suggerisce che il testo potrebbe essere ambiguo o contenere sentiment misti.

Il modello può sbagliare quando:
- Il testo contiene sarcasmo o ironia
- Il contesto è ambiguo o mancano informazioni
- Il linguaggio è molto tecnico o settoriale
- Ci sono errori di ortografia significativi

Per questo motivo, in produzione è importante monitorare la distribuzione dei sentiment nel tempo e rilevare quando cambia significativamente rispetto alla baseline storica.


## 4. Architettura del Progetto

Il sistema è organizzato in diversi componenti che lavorano insieme per fornire una soluzione completa di monitoring della reputazione.

### Flusso End-to-End

```
Feedback Social (LinkedIn, Forum, etc.)
    ↓
FastAPI API (/predict, /predict/batch)
    ↓
Modello Hugging Face (sentiment analysis)
    ↓
Metriche Prometheus (/metrics)
    ↓
Prometheus (scraping ogni 5 secondi)
    ↓
Grafana Dashboard (visualizzazione)
```

### Componenti Principali

**FastAPI**: Framework web moderno per servire il modello come API REST. Offre validazione automatica con Pydantic, documentazione auto-generata e performance asincrone.

**Pytest**: Suite di test automatici che verifica il corretto funzionamento degli endpoint e della logica di inferenza. I test vengono eseguiti automaticamente ad ogni commit tramite GitHub Actions.

**Docker + Docker Compose**: Containerizzazione dell'applicazione per deployment consistente. Docker Compose orchestra tre servizi: API, Prometheus e Grafana.

**CI/CD con GitHub Actions**: Pipeline automatizzata che esegue test ad ogni push e build/push dell'immagine Docker su GitHub Container Registry ad ogni release.

**Prometheus + Grafana**: Sistema di monitoring che raccoglie metriche operative (request rate, latenza, distribuzione sentiment) e le visualizza su dashboard interattive.

**Drift Detection**: Implementazione di KL divergence per rilevare quando la distribuzione dei sentiment cambia rispetto a una baseline, segnalando possibili problemi o cambiamenti nel comportamento dei clienti.


## 5. Estratti di Codice Significativi

Di seguito alcuni estratti di codice che mostrano le scelte tecniche principali del progetto.


### Endpoint FastAPI con Validazione Pydantic

L'endpoint `/predict` utilizza Pydantic per validare automaticamente l'input. Se il testo è vuoto o troppo lungo, FastAPI restituisce un errore 422 senza eseguire l'inferenza.


In [ ]:
# Esempio di schema Pydantic e endpoint FastAPI
from pydantic import BaseModel, Field
from fastapi import FastAPI

class TextItem(BaseModel):
    """Schema per richiesta di predizione singola."""
    text: str = Field(..., min_length=1, max_length=1000)

class Prediction(BaseModel):
    """Schema per risposta."""
    label: str
    score: float = Field(..., ge=0.0, le=1.0)

app = FastAPI()

@app.post("/predict", response_model=Prediction)
async def predict(item: TextItem):
    """Predice il sentiment per un singolo testo."""
    # La validazione è automatica: testo vuoto o > 1000 caratteri → 422
    # In produzione qui chiameremmo predict_one(item.text)
    return Prediction(label="positive", score=0.95)

print("Schema Pydantic definito: validazione automatica di input/output")


### Dockerfile

Il Dockerfile crea un'immagine leggera partendo da `python:3.10-slim`. Le dipendenze vengono installate prima del codice sorgente per sfruttare la cache di Docker e velocizzare rebuild successivi.


In [ ]:
dockerfile_content = """
# Dockerfile per sentiment reputation monitoring API
FROM python:3.10-slim

WORKDIR /app

RUN apt-get update && apt-get install -y --no-install-recommends \\
    && rm -rf /var/lib/apt/lists/*

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY src/ ./src/

EXPOSE 8000

CMD ["uvicorn", "src.app.main:app", "--host", "0.0.0.0", "--port", "8000"]
"""

print("Dockerfile:")
print(dockerfile_content)


### Metriche Prometheus

Le metriche vengono esposte in formato Prometheus compatibile. Un Counter traccia il numero totale di richieste, un Histogram misura la latenza, e un Counter separato traccia la distribuzione dei sentiment.


In [ ]:
from prometheus_client import Counter, Histogram

# Definizione metriche Prometheus
REQUEST_COUNT = Counter(
    'fastapi_requests_total',
    'Total number of requests',
    ['method', 'endpoint', 'status']
)

REQUEST_LATENCY = Histogram(
    'fastapi_request_latency_seconds',
    'Request latency in seconds',
    ['method', 'endpoint'],
    buckets=[0.01, 0.05, 0.1, 0.5, 1.0, 2.0, 5.0]
)

SENTIMENT_DISTRIBUTION = Counter(
    'sentiment_predictions_total',
    'Total sentiment predictions by label',
    ['label']
)

print("Metriche Prometheus definite:")
print("- fastapi_requests_total: conta richieste per metodo/endpoint/status")
print("- fastapi_request_latency_seconds: misura latenza con bucket predefiniti")
print("- sentiment_predictions_total: conta predizioni per label")


### Test Pytest

I test verificano che gli endpoint rispondano correttamente e che la validazione funzioni. Usiamo `TestClient` di FastAPI per simulare richieste HTTP senza avviare un server reale.


In [ ]:
test_code = '''
from fastapi.testclient import TestClient
from src.app.main import app

client = TestClient(app)

def test_predict_endpoint():
    """Test endpoint /predict."""
    response = client.post(
        "/predict",
        json={"text": "I love this product!"}
    )
    
    assert response.status_code == 200
    data = response.json()
    assert "label" in data
    assert "score" in data
    assert data["label"] in ["negative", "neutral", "positive"]
    assert 0.0 <= data["score"] <= 1.0

def test_predict_endpoint_invalid():
    """Test validazione input."""
    response = client.post("/predict", json={"text": ""})
    assert response.status_code == 422  # Validation error
'''

print("Esempio test Pytest:")
print(test_code)


### Pipeline CI/CD con GitHub Actions

Il workflow CI si attiva su ogni pull request e push su main. Esegue i test automaticamente e builda l'immagine Docker per validazione.


In [ ]:
ci_workflow = '''
name: CI

on:
  pull_request:
    branches: [main, master]
  push:
    branches: [main, master]

jobs:
  test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v4
      - uses: actions/setup-python@v5
        with:
          python-version: "3.10"
      - run: pip install -r requirements.txt
      - run: pytest --cov=src
      - run: docker build -f docker/Dockerfile -t test-image .
'''

print("Workflow CI GitHub Actions:")
print(ci_workflow)


## 6. Risultati e Test

I test del progetto coprono tre aree principali: test unitari per le funzioni di inferenza, test di integrazione per gli endpoint API, e test di health check. Tutti i test passano correttamente.

### Output Test

Eseguendo `pytest` sul progetto completo si ottiene:

```
============================= test session starts ==============================
collected 12 items

tests/test_api_integration.py .....                                      [ 41%]
tests/test_health.py .                                                   [ 50%]
tests/test_infer_unit.py ......                                          [100%]

======================== 12 passed, 3 warnings in 6.46s ========================
```

### Esempio Output Endpoint

**GET /health:**
```json
{
  "status": "ok",
  "model_loaded": true,
  "model_name": "cardiffnlp/twitter-roberta-base-sentiment-latest"
}
```

**POST /predict:**
```json
{
  "label": "positive",
  "score": 0.9382082223892212
}
```

**GET /metrics:**
Output in formato Prometheus exposition format con metriche come:
- `fastapi_requests_total{method="POST",endpoint="/predict",status="200"} 42`
- `fastapi_request_latency_seconds_bucket{method="POST",endpoint="/predict",le="0.5"} 38`
- `sentiment_predictions_total{label="positive"} 25`


In [ ]:
# Simulazione di chiamate API per dimostrazione
print("=== Simulazione Chiamate API ===\n")

# Health check
print("1. GET /health")
print('   Response: {"status": "ok", "model_loaded": true, "model_name": "cardiffnlp/twitter-roberta-base-sentiment-latest"}')
print()

# Predict
print("2. POST /predict")
print('   Request: {"text": "Great quality gears!"}')
print('   Response: {"label": "positive", "score": 0.94}')
print()

# Metrics
print("3. GET /metrics")
print("   Response: Formato Prometheus con metriche esposte")
print("   - fastapi_requests_total")
print("   - fastapi_request_latency_seconds")
print("   - sentiment_predictions_total")


## 7. Sviluppi Futuri e Miglioramenti

Il sistema attuale funziona bene per un caso d'uso base, ma ci sono diverse aree di miglioramento che potrebbero essere implementate in futuro.

**Fine-tuning del modello**: Il modello pre-addestrato funziona bene su testi generici, ma potrebbe essere migliorato con un fine-tuning su feedback specifici del settore meccanico. Questo richiederebbe la raccolta di un dataset etichettato di feedback reali dell'azienda.

**Integrazione con fonti dati reali**: Attualmente il sistema riceve testi via API, ma in produzione servirebbe integrazione diretta con API di social media (LinkedIn, Twitter) o scraping periodico di forum e recensioni. Questo richiederebbe gestione di autenticazione, rate limiting e parsing di formati diversi.

**Rate limiting e sicurezza**: Per proteggere l'API da abusi, servirebbe implementare rate limiting per IP e autenticazione con API key. FastAPI supporta middleware per questo scopo.

**Caching delle predizioni**: Testi identici o molto simili vengono riprocessati ogni volta. Un sistema di caching (es. Redis) potrebbe ridurre la latenza e il carico sul modello per richieste frequenti.

**Dashboard Grafana più dettagliata**: La dashboard attuale mostra metriche base. Potrebbe essere estesa con alerting automatico quando il drift supera una soglia, visualizzazioni di trend storici, e correlazione con eventi esterni (es. lanci di prodotto, campagne marketing).

**Versioning del modello**: Quando si fa retraining o si cambia modello, servirebbe un sistema per gestire versioni multiple e fare A/B testing prima di sostituire completamente il modello in produzione.

**Retraining automatico**: Un DAG Airflow potrebbe schedulare retraining periodico (es. mensile) usando nuovi dati raccolti, validare le performance su un test set, e deployare automaticamente se le metriche migliorano.


## 8. Conclusioni

Questo progetto dimostra come costruire una pipeline MLOps completa per un caso d'uso reale di sentiment analysis. Abbiamo visto come integrare un modello Hugging Face pre-addestrato in un'API FastAPI, come esporre metriche per monitoring, come containerizzare l'applicazione con Docker, e come automatizzare test e deployment con CI/CD.

Il sistema è utile perché trasforma dati non strutturati (feedback testuali) in metriche operative monitorabili, permettendo all'azienda di reagire rapidamente a cambiamenti nella percezione del brand o a problemi emergenti. Il monitoring continuo e il drift detection aiutano a mantenere il sistema affidabile nel tempo.

L'approccio modulare e containerizzato rende il sistema facilmente deployabile e scalabile. La pipeline CI/CD garantisce che ogni modifica sia testata automaticamente prima di essere rilasciata in produzione.

Il codice completo, inclusi test, configurazione Docker, workflow CI/CD e documentazione, è disponibile sul repository GitHub al link indicato nella sezione Introduzione.
